In [111]:
import pandas as pd
df = pd.read_csv('Papers for Labeling-Grid view.csv')
columns = df.columns

In [112]:
df

,DOI,Abstract,Journal,Journal URL,PubMed URL,Open access?,Press release
0,10.1140/epjst/e2020-900273-x,Cold-weather penguins continually dive in and ...,The European Physical Journal Special Topics,https://link.springer.com/article/10.1140/epjs...,NaN,NaN,https://www.sciencedaily.com/releases/2015/11/...
1,10.1016/j.jpowsour.2019.01.027,The discovery of the ability of microorganisms...,Journal of Power Sources,https://www.sciencedirect.com/science/article/...,NaN,NaN,https://www.sciencedaily.com/releases/2019/03/...
2,10.1111/jav.01972,Flight activity recorders have recently confir...,Journal of Avian Biology,https://onlinelibrary.wiley.com/doi/abs/10.111...,NaN,NaN,https://www.sciencedaily.com/releases/2019/03/...
3,10.1137/19M1262322,We consider the inverse problem of localizing ...,Society for Industrial and Applied Mathematics,https://epubs.siam.org/doi/10.1137/19M1262322,NaN,NaN,https://www.sciencedaily.com/releases/2019/12/...
4,10.5286/ISIS.E.92924017,Cyphochilus beetle scales are amongst the brig...,Science,https://www.nature.com/articles/s42004-019-020...,NaN,NaN,NaN
...,...,...,...,...,...,...,...
338,NaN,NaN,NaN,NaN,NaN,NaN,https://www.sciencedaily.com/releases/2020/10/...
339,NaN,NaN,NaN,NaN,NaN,NaN,https://iopscience.iop.org/article/10.1088/174...
340,NaN,NaN,NaN,NaN,NaN,NaN,https://iopscience.iop.org/article/10.1088/174...
341,NaN,NaN,NaN,NaN,NaN,NaN,https://royalsocietypublishing.org/doi/10.1098...


In [106]:
#df = df[df['Journal URL'].notna()]
df.iloc[38]['Journal URL']

'https://www.tandfonline.com/doi/abs/10.1080/00033796800200111 https://www.nature.com/articles/ncomms7368'

In [116]:
def clean_airtable_v2(df):
    rows_lst = []
    for index, row in df.iterrows():
        if pd.isnull(row['Journal URL']):
            rows_lst.append(list(row))
            continue
        rows_lst.append(list(row))
        #Retrieving list of URLS, and only operating if it has multiple
        urls = row['Journal URL'].split(' ')
    
        if len(urls) >= 2:
            rows_lst.remove(list(row))
            #If it has multiple URLS, drop that row for now
            #df = df.drop(index=index)
            #Create new row for each URL with same data in all other columns
            for url in urls:
                if len(url) > 2:
                    data = [row['DOI'], row['Abstract'], row['Journal'], url,
                    row['PubMed URL'], row['Open access?'], row['Press release']
                    ]
                    #Append new rows to dataframe
                    rows_lst.append(data)
    return pd.DataFrame(rows_lst, columns = df.columns).drop_duplicates()
clean_df = clean_airtable_v2(df)

In [ ]:
import requests
from bs4 import BeautifulSoup

def press_release_link(row):
    url = row['Journal URL']
    press_release = row['Press release']
    if pd.isnull(url) and not pd.isnull(press_release):
        r = requests.get(press_release)
        soup = BeautifulSoup(r.text)
        
        

In [87]:
def clean_multiple_urls(df):
    rows_lst = []
    for index, row in df.iterrows():
        rows_lst.append(list(row))
        #Retrieving list of URLS, and only operating if it has multiple
        urls = row['Primary lit site'].split('\n')
    
        if len(urls) >= 2:
            rows_lst.remove(list(row))
            #If it has multiple URLS, drop that row for now
            #df = df.drop(index=index)
            #Create new row for each URL with same data in all other columns
            for url in urls:
                if len(url) > 2:
                    data = [row['DOI'], row['Paper title'], row['Abstract'], row['Journal'], url,
                    row['Functions Level I'], row['Functions Level II'], row['Functions Level III- NEW'],
                    row['Functions Level III-OLD'], row['Link to press release']
                    ]
                    #Append new rows to dataframe
                    rows_lst.append(data)
    return pd.DataFrame(rows_lst).drop_duplicates()

df = clean_multiple_urls(df)

In [88]:
df.columns = columns

In [90]:
len(df)

1375

In [91]:
for url in df['Primary lit site'].values:
    if len(url.split(' ')) > 2:
        print(u)

In [92]:
df = df[df['Primary lit site'].notna()]
len(df)

1375

In [93]:
df = df.reset_index()

In [94]:
df.to_csv('cleaned_airtable.csv')